# TCGA Random Forest Machine Learning

In [1]:
# Import Libraries
import gzip
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
from scipy import stats
from pprint import pprint

# Import PCA libraries
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from IPython.display import Image
import umap
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Import ML libraries

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc



print("sucess!")

sucess!


## Load in our data

In [2]:
# Load in Data + Trnasform Direction
df=pd.read_csv('COAD_normalized_counts.csv') # load
df=df.transpose()

genes=df.loc['new_gene'].to_list() # saves all genes as list
df.columns=genes

samples=df.index.to_list()
samples=samples[1:] # removes the "New_Gene" Label

df=df.drop(['new_gene']).reset_index(drop=True) # remove gene column 
df.index=samples # set index names 

df=df.astype(float) # weird nonsense step thats needed ~sometimes~
df.head() # show results

KeyError: 'new_gene'

In [4]:
# Pre-Process: Filter out low expressed genes
count=0; # keep track of how many genese removed
thresh=25 # min number of samples needed
for col in df.columns.to_list():
    n_zeros=df[col].isin([0]).sum()    
    if n_zeros >= thresh:
        count+=1
        df.drop([col],axis=1,inplace=True)
genes=df.columns.to_list()
print("Removed %d genes. These genes did not meet 10 sample mininum. \nWe have %d genes left \
"%(count,len(df.columns.to_list())))


# log transform data
df_log=pd.DataFrame()
for col in df.columns.to_list():
    df_log[col] = df[col].apply(lambda x: np.log(x+1))
df_log.head()


# gene signature
df_gene_sig=df_log[['TREML2', 'PTPRN', 'PGLYRP1', 'NOG', 'VIP',\
    'RIMKLB', 'NKAIN4', 'FAM171B', 'ZNF417', 'GLRA2', 'HOXA7', 'FABP6', 'MUSK',\
    'HTR6', 'GRIP2', 'VEGFA', 'AKAP12', 'RHEB', 'PMEPA1','GLTP', 'METTL7A',\
    'CITED2', 'SCARA5', 'CDH3','IL6R', 'PKIB', 'GLP2R', 'EPB41L3', 'NR3C2']]

# Genes not found from gene signature (possible naming issues)
# PADI4, NCKIPSD C5orf53 TREML3 HES5 OR8D2 KLRK1 NCRNA00152 PPAP2A LINC00974


df_gene_sig.head(10)

Removed 0 genes. These genes did not meet 10 sample mininum. 
We have 16893 genes left 


KeyError: "None of [Index(['TREML2', 'PTPRN', 'PGLYRP1', 'NOG', 'VIP', 'RIMKLB', 'NKAIN4',\n       'FAM171B', 'ZNF417', 'GLRA2', 'HOXA7', 'FABP6', 'MUSK', 'HTR6', 'GRIP2',\n       'VEGFA', 'AKAP12', 'RHEB', 'PMEPA1', 'GLTP', 'METTL7A', 'CITED2',\n       'SCARA5', 'CDH3', 'IL6R', 'PKIB', 'GLP2R', 'EPB41L3', 'NR3C2'],\n      dtype='object')] are in the [columns]"

## Extract tissue sample from columns (healthy or tumor)

In [20]:
# Create Target vector
target=[]
sample_names=[]

# loop through and extract tumor + healthy
for i in samples:
    
    temp=i.split('_')[1] # Split string using "_"
    #remove numbers at end (if they have it)
    if temp[-1].isnumeric():
        temp=temp[:-1]
    
    target.append(temp)# save tissues type to list (in order)

        
# Convert to Panda Series
target=pd.Series(target,name='Tissue')

0      Tumor
1    Healthy
2    Healthy
3      Tumor
4    Healthy
Name: Tissue, dtype: object

In [19]:
# Convert to binary tumor=(1) + healthy=(0)
target_binary=[]
for i in target:
    if i=='Healthy':
        target_binary.append(0)
    else:
        target_binary.append(1)
        
# Convert to Panda Series
target_binary=pd.Series(target_binary)

0    1
1    0
2    0
3    1
4    0
dtype: int64

In [8]:
# create colors labels for PCA
colors=[]
for i in target_binary:
    if i ==1:
        colors.append('orange')
    else:
        colors.append('blue')

## Vizualize data in a PCA Plot
https://towardsdatascience.com/explaining-k-means-clustering-5298dc47bad6

In [1]:
# Generate Colors + numbers
num_list=range(0,54)

# PCA 
pca = PCA(n_components=2)
X_r = pca.fit(df_log).transform(df_log)
variance = pca.explained_variance_ratio_


# Plot
for color, i, target_name in zip(colors, num_list, samples):
    plt.scatter(X_r[i,0], X_r[i,1], color=color, alpha=.8, lw=2,
                label=target_name)
    
plt.title('PCA of Samples Using All Genes', fontsize=18, fontweight='black', color = '#333F4B')
plt.axvline(x=0,linestyle='--',c='k')
plt.axhline(y=0,linestyle='--',c='k')
plt.tick_params(axis='y', which='major', labelsize=14)
plt.tick_params(axis='x', which='major', labelsize=14)
plt.grid(True)
plt.xlabel('Component 1', fontsize=14, fontweight='black', color = '#333F4B')
plt.ylabel('Component 2', fontsize=14, fontweight='black', color = '#333F4B')


NameError: name 'PCA' is not defined

## Splitting data for Cross Validation

Split into testing and training data

In [36]:
# Split Into Training and Test Data

# independent dataset : df_gene_sig : features that determine if patient has cancer
# dependent dataset : target_bianry : diagnosis

x_train, x_test, y_train, y_test = train_test_split(df_gene_sig, target_binary, test_size=15,shuffle=True, random_state=12)
# display size of 
print('Our Training Data Set has %d patient samples'%(np.shape(x_train)[0]))
print('Our Testing Data Set has %d patient samples'%(np.shape(x_test)[0]))

Our Training Data Set has 40 patient samples
Our Testing Data Set has 15 patient samples


## Random Forest Model
https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

Train our data.

In [81]:
# X =df_gene_sig, y = target_binary,
# x_train, x_test, y_train, y_test
print('\nTrain Data:')

# Initialize random forest mode
rfc = RandomForestClassifier()

# Train model on training data
rfc.fit(x_train, y_train)

# Get the reults of the training data
results = rfc.predict(x_train)

cm=confusion_matrix(y_train, results)
fpr_rfc, tpr_rfc, thresholds_rfc = roc_curve(y_train, results)
auc_k = auc(fpr_rfc, tpr_rfc)

plt.plot(fpr_kera, tpr_kera, label=lab)
acc=round((cm[1,1]+cm[0,0])/45,4)
print(cm)
print('Accuracy:',acc)
print('AUC: %.4f'%(auc_k))
acc_tot=acc_tot+acc



Test Data

In [ ]:
# Test Set - now that we've trained, we can now use test set to
print('\nTest Data:')

# run it foward - to predict using TEST
results=estimator.predict(x_test)

# Create Confusion matrix using prediction + truth 
cm=confusion_matrix(y_test, results)
# Generate ROC Curve 
fpr_kera, tpr_kera, thresholds_rf = roc_curve(y_test, results)
# Calculate AUC (Area under the ROC Curve )
auc_k = auc(fpr_kera, tpr_kera)

# label for plot
lab='Test'
plt.plot(fpr_kera, tpr_kera, label=lab)

# show results FOR TEST + Overall Train
print(cm)
print('Accuracy:',round((cm[1,1]+cm[0,0])/15,4))
print('AUC: %.4f'%(auc_k))
print('\nOverall Acc is:\n%.4f'%(acc_tot/4))

# plot adjustment
plt.legend()
plt.plot([0, 1], [0, 1],'r--',linewidth=2.5)

## Cross Validation

In [79]:
def plot_roc_curve(fprs, tprs):
    """Plot the Receiver Operating Characteristic from a list
    of true positive rates and false positive rates."""
    
    # Initialize useful lists + the plot axes.
    tprs_interp = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    f, ax = plt.subplots(figsize=(14,10))
    
    # Plot ROC for each K-Fold + compute AUC scores.
    for i, (fpr, tpr) in enumerate(zip(fprs, tprs)):
        tprs_interp.append(np.interp(mean_fpr, fpr, tpr))
        tprs_interp[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        ax.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        
    # Plot the luck line.
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Luck', alpha=.8)
    
    # Plot the mean ROC.
    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)
    
    # Plot the standard deviation around the mean ROC.
    std_tpr = np.std(tprs_interp, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'$\pm$ 1 std. dev.')
    
    # Fine tune and show the plot.
    #ax.set_xlim([-0.05, 1.05])
    #ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right")
    plt.show()
    return (f, ax)

def compute_roc_auc(index):
    y_predict = rfc.predict_proba(X.iloc[index])[:,1]
    fpr, tpr, thresholds = roc_curve(y.iloc[index], y_predict)
    auc_score = auc(fpr, tpr)
    return fpr, tpr, auc_score

In [70]:
# Intialize counter + Total Accuarcy
c=1
acc_tot=0

for train_index, test_index in skf.split(x_train, y_train):
    print('\nk-Fold Number: %d'%(c))
    
    # Get nth fold data (from before where we print out index)
    X_train = x_train.iloc[train_index]
    Y_train = y_train.iloc[train_index]

    # Train model using fold data
    rfc.fit(X_train,Y_train)
    # predictions
    rfc_predict = rfc.predict(x_test)
    
    
    
    
    
    
# Evaluate Performancerst
rfc_cv_score = cross_val_score(rfc, df_gene_sig, target_binary, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())
    


k-Fold Number: 1
=== Confusion Matrix ===
[[3 2]
 [3 7]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.50      0.60      0.55         5
           1       0.78      0.70      0.74        10

    accuracy                           0.67        15
   macro avg       0.64      0.65      0.64        15
weighted avg       0.69      0.67      0.67        15



=== All AUC Scores ===
[0.75       0.88888889 0.55555556 0.33333333 0.33333333 0.91666667
 0.66666667 0.83333333 0.33333333 0.66666667]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.6277777777777778

k-Fold Number: 1
=== Confusion Matrix ===
[[3 2]
 [2 8]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.60      0.60      0.60         5
           1       0.80      0.80      0.80        10

    accuracy                           0.73        15
   macro avg       0.70      0.70      0.70        15
weight

In [ ]:
feature_importances = pd.DataFrame(rf.feature_importances_, index =rf.columns,  columns=['importance']).sort_values('importance', ascending=False)